# Task 3 - Modeling

This notebook will get you started by helping you to load the data, but then it'll be up to you to complete the task! If you need help, refer to the `modeling_walkthrough.ipynb` notebook.


## Section 1 - Setup

First, we need to mount this notebook to our Google Drive folder, in order to access the CSV data file.


In [269]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [270]:
!pip install datetime_truncate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


We want to use dataframes once again to store and manipulate the data.

In [271]:
# Importing all the required libraries

import os
import pandas as pd
import numpy as np
import datetime as dt
import datetime_truncate as dtr
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import Normalizer
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.datasets import make_regression
from sklearn.svm import SVR
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_validate
import joblib
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

---

## Section 2 - Data loading

Similar to before, let's load our data from Google Drive for the 3 datasets provided. Be sure to upload the datasets into Google Drive, so that you can access them here.

In [272]:
path = "/content/drive/MyDrive/"

sales_df = pd.read_csv(f"{path}sales.csv")
sales_df.drop(columns=["Unnamed: 0"], inplace=True, errors='ignore')
sales_df.head()

,transaction_id,timestamp,product_id,category,customer_type,unit_price,quantity,total,payment_type
0,a1c82654-c52c-45b3-8ce8-4c2a1efe63ed,2022-03-02 09:51:38,3bc6c1ea-0198-46de-9ffd-514ae3338713,fruit,gold,3.99,2,7.98,e-wallet
1,931ad550-09e8-4da6-beaa-8c9d17be9c60,2022-03-06 10:33:59,ad81b46c-bf38-41cf-9b54-5fe7f5eba93e,fruit,standard,3.99,1,3.99,e-wallet
2,ae133534-6f61-4cd6-b6b8-d1c1d8d90aea,2022-03-04 17:20:21,7c55cbd4-f306-4c04-a030-628cbe7867c1,fruit,premium,0.19,2,0.38,e-wallet
3,157cebd9-aaf0-475d-8a11-7c8e0f5b76e4,2022-03-02 17:23:58,80da8348-1707-403f-8be7-9e6deeccc883,fruit,gold,0.19,4,0.76,e-wallet
4,a81a6cd3-5e0c-44a2-826c-aea43e46c514,2022-03-05 14:32:43,7f5e86e6-f06f-45f6-bf44-27b095c9ad1d,fruit,basic,4.49,2,8.98,debit card


In [273]:
stock_df = pd.read_csv(f"{path}sensor_stock_levels.csv")
stock_df.drop(columns=["Unnamed: 0"], inplace=True, errors='ignore')
stock_df.head()

,id,timestamp,product_id,estimated_stock_pct
0,4220e505-c247-478d-9831-6b9f87a4488a,2022-03-07 12:13:02,f658605e-75f3-4fed-a655-c0903f344427,0.75
1,f2612b26-fc82-49ea-8940-0751fdd4d9ef,2022-03-07 16:39:46,de06083a-f5c0-451d-b2f4-9ab88b52609d,0.48
2,989a287f-67e6-4478-aa49-c3a35dac0e2e,2022-03-01 18:17:43,ce8f3a04-d1a4-43b1-a7c2-fa1b8e7674c8,0.58
3,af8e5683-d247-46ac-9909-1a77bdebefb2,2022-03-02 14:29:09,c21e3ba9-92a3-4745-92c2-6faef73223f7,0.79
4,08a32247-3f44-4002-85fb-c198434dd4bb,2022-03-02 13:46:18,7f478817-aa5b-44e9-9059-8045228c9eb0,0.22


In [274]:
temp_df = pd.read_csv(f"{path}sensor_storage_temperature.csv")
temp_df.drop(columns=["Unnamed: 0"], inplace=True, errors='ignore')
temp_df.head()

,id,timestamp,temperature
0,d1ca1ef8-0eac-42fc-af80-97106efc7b13,2022-03-07 15:55:20,2.96
1,4b8a66c4-0f3a-4f16-826f-8cf9397e9d18,2022-03-01 09:18:22,1.88
2,3d47a0c7-1e72-4512-812f-b6b5d8428cf3,2022-03-04 15:12:26,1.78
3,9500357b-ce15-424a-837a-7677b386f471,2022-03-02 12:30:42,2.18
4,c4b61fec-99c2-4c6d-8e5d-4edd8c9632fa,2022-03-05 09:09:33,1.38


Now it's up to you, refer back to the steps in your strategic plan to complete this task. Good luck!

In [275]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7829 entries, 0 to 7828
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction_id  7829 non-null   object 
 1   timestamp       7829 non-null   object 
 2   product_id      7829 non-null   object 
 3   category        7829 non-null   object 
 4   customer_type   7829 non-null   object 
 5   unit_price      7829 non-null   float64
 6   quantity        7829 non-null   int64  
 7   total           7829 non-null   float64
 8   payment_type    7829 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 550.6+ KB


In [276]:
stock_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   15000 non-null  object 
 1   timestamp            15000 non-null  object 
 2   product_id           15000 non-null  object 
 3   estimated_stock_pct  15000 non-null  float64
dtypes: float64(1), object(3)
memory usage: 468.9+ KB


In [277]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23890 entries, 0 to 23889
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           23890 non-null  object 
 1   timestamp    23890 non-null  object 
 2   temperature  23890 non-null  float64
dtypes: float64(1), object(2)
memory usage: 560.0+ KB


In [278]:
sales_df.timestamp.value_counts()

2022-03-02 19:32:20    2
2022-03-05 10:11:33    2
2022-03-01 18:12:17    2
2022-03-03 13:28:14    2
2022-03-07 17:13:29    2
                      ..
2022-03-01 10:00:58    1
2022-03-02 11:05:58    1
2022-03-04 17:48:49    1
2022-03-04 14:53:27    1
2022-03-06 13:50:36    1
Name: timestamp, Length: 7738, dtype: int64

In [279]:
!pip install --upgrade pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Section 3: Preprocessing the Data


In [280]:
#Convert timestamp to Hour Period
sales_df['timestamp'] = sales_df.timestamp.apply(lambda x: dtr.truncate_hour(pd.to_datetime(x)))
stock_df['timestamp'] = stock_df.timestamp.apply(lambda x: dtr.truncate_hour(pd.to_datetime(x)))
temp_df['timestamp'] = temp_df.timestamp.apply(lambda x: dtr.truncate_hour(pd.to_datetime(x)))


In [281]:
sales_df['timestamp']

0      2022-03-02 09:00:00
1      2022-03-06 10:00:00
2      2022-03-04 17:00:00
3      2022-03-02 17:00:00
4      2022-03-05 14:00:00
               ...        
7824   2022-03-03 18:00:00
7825   2022-03-04 19:00:00
7826   2022-03-03 14:00:00
7827   2022-03-04 15:00:00
7828   2022-03-06 13:00:00
Name: timestamp, Length: 7829, dtype: datetime64[ns]

In [282]:
# Group by hour and product
sales_grped = sales_df.groupby(['timestamp', 'product_id']).agg({'quantity': 'sum', 'total': 'mean'}).reset_index()
stock_grped = stock_df.groupby(['timestamp', 'product_id']).agg({'estimated_stock_pct': 'mean'}).reset_index()
temp_grped = temp_df.groupby(['timestamp']).agg({'temperature': 'mean'}).reset_index()
sales_grped.head()

,timestamp,product_id,quantity,total
0,2022-03-01 09:00:00,00e120bb-89d6-4df5-bc48-a051148e3d03,3,33.5700
1,2022-03-01 09:00:00,01f3cdd9-8e9e-4dff-9b5c-69698a0388d0,3,4.4700
2,2022-03-01 09:00:00,03a2557a-aa12-4add-a6d4-77dc36342067,3,17.9700
3,2022-03-01 09:00:00,049b2171-0eeb-4a3e-bf98-0c290c7821da,7,8.7150
4,2022-03-01 09:00:00,04da844d-8dba-4470-9119-e534d52a03a0,11,1.3475


In [283]:
stock_grped.head()

,timestamp,product_id,estimated_stock_pct
0,2022-03-01 09:00:00,00e120bb-89d6-4df5-bc48-a051148e3d03,0.89
1,2022-03-01 09:00:00,01f3cdd9-8e9e-4dff-9b5c-69698a0388d0,0.14
2,2022-03-01 09:00:00,01ff0803-ae73-4234-971d-5713c97b7f4b,0.67
3,2022-03-01 09:00:00,0363eb21-8c74-47e1-a216-c37e565e5ceb,0.82
4,2022-03-01 09:00:00,03f0b20e-3b5b-444f-bc39-cdfa2523d4bc,0.05


In [284]:
temp_grped.head()

,timestamp,temperature
0,2022-03-01 09:00:00,-0.028850
1,2022-03-01 10:00:00,1.284314
2,2022-03-01 11:00:00,-0.560000
3,2022-03-01 12:00:00,-0.537721
4,2022-03-01 13:00:00,-0.188734


In [285]:
#Merging the diffrent DataFrames
merged_df = stock_grped.merge(sales_grped, on=['timestamp', 'product_id'], how='left') \
       .merge(temp_grped, on='timestamp', how='left')
merged_df.head()

,timestamp,product_id,estimated_stock_pct,quantity,total,temperature
0,2022-03-01 09:00:00,00e120bb-89d6-4df5-bc48-a051148e3d03,0.89,3.0,33.57,-0.02885
1,2022-03-01 09:00:00,01f3cdd9-8e9e-4dff-9b5c-69698a0388d0,0.14,3.0,4.47,-0.02885
2,2022-03-01 09:00:00,01ff0803-ae73-4234-971d-5713c97b7f4b,0.67,NaN,NaN,-0.02885
3,2022-03-01 09:00:00,0363eb21-8c74-47e1-a216-c37e565e5ceb,0.82,NaN,NaN,-0.02885
4,2022-03-01 09:00:00,03f0b20e-3b5b-444f-bc39-cdfa2523d4bc,0.05,NaN,NaN,-0.02885


In [286]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10845 entries, 0 to 10844
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   timestamp            10845 non-null  datetime64[ns]
 1   product_id           10845 non-null  object        
 2   estimated_stock_pct  10845 non-null  float64       
 3   quantity             3067 non-null   float64       
 4   total                3067 non-null   float64       
 5   temperature          10845 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 593.1+ KB


In [287]:
# Adding the Product features in our Main Dataframe
category_feature = sales_df[['product_id', 'category']]
category_feature.drop_duplicates(inplace=True)

product_price = sales_df[['product_id', 'unit_price']]
product_price.drop_duplicates(inplace=True)

In [288]:
# Merging it into the Main Dataframe
merged_df = merged_df.merge(category_feature, on='product_id', how='left') \
            .merge(product_price, on='product_id', how='left')
merged_df.head()

,timestamp,product_id,estimated_stock_pct,quantity,total,temperature,category,unit_price
0,2022-03-01 09:00:00,00e120bb-89d6-4df5-bc48-a051148e3d03,0.89,3.0,33.57,-0.02885,kitchen,11.19
1,2022-03-01 09:00:00,01f3cdd9-8e9e-4dff-9b5c-69698a0388d0,0.14,3.0,4.47,-0.02885,vegetables,1.49
2,2022-03-01 09:00:00,01ff0803-ae73-4234-971d-5713c97b7f4b,0.67,NaN,NaN,-0.02885,baby products,14.19
3,2022-03-01 09:00:00,0363eb21-8c74-47e1-a216-c37e565e5ceb,0.82,NaN,NaN,-0.02885,beverages,20.19
4,2022-03-01 09:00:00,03f0b20e-3b5b-444f-bc39-cdfa2523d4bc,0.05,NaN,NaN,-0.02885,pets,8.19


In [289]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10845 entries, 0 to 10844
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   timestamp            10845 non-null  datetime64[ns]
 1   product_id           10845 non-null  object        
 2   estimated_stock_pct  10845 non-null  float64       
 3   quantity             3067 non-null   float64       
 4   total                3067 non-null   float64       
 5   temperature          10845 non-null  float64       
 6   category             10845 non-null  object        
 7   unit_price           10845 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(2)
memory usage: 762.5+ KB


In [290]:
#As we can see that the "quantity" and "total" has some Null Values hence we will fill NA with Zeroes.

merged_df.quantity.fillna(0, inplace=True)
merged_df.total.fillna(0, inplace=True)

##Feature Engineering

In [291]:
merged_df.corr()

,estimated_stock_pct,quantity,total,temperature,unit_price
estimated_stock_pct,1.000000,0.012929,0.004200,0.007955,-0.024479
quantity,0.012929,1.000000,0.658054,-0.017771,-0.108101
total,0.004200,0.658054,1.000000,-0.017438,0.245598
temperature,0.007955,-0.017771,-0.017438,1.000000,0.002874
unit_price,-0.024479,-0.108101,0.245598,0.002874,1.000000


In [292]:
#  Extract other date features like hour, dayofweek, month and day.
merged_df['month'] = merged_df.timestamp.dt.month
merged_df['day'] = merged_df.timestamp.dt.day
merged_df['day_of_week'] = merged_df.timestamp.dt.dayofweek
merged_df['hour'] = merged_df.timestamp.dt.hour

merged_df.drop(['timestamp', 'product_id'], axis=1, inplace=True)
merged_df

,estimated_stock_pct,quantity,total,temperature,category,unit_price,month,day,day_of_week,hour
0,0.89,3.0,33.57,-0.028850,kitchen,11.19,3,1,1,9
1,0.14,3.0,4.47,-0.028850,vegetables,1.49,3,1,1,9
2,0.67,0.0,0.00,-0.028850,baby products,14.19,3,1,1,9
3,0.82,0.0,0.00,-0.028850,beverages,20.19,3,1,1,9
4,0.05,0.0,0.00,-0.028850,pets,8.19,3,1,1,9
...,...,...,...,...,...,...,...,...,...,...
10840,0.50,4.0,19.96,-0.165077,fruit,4.99,3,7,0,19
10841,0.26,0.0,0.00,-0.165077,meat,19.99,3,7,0,19
10842,0.78,3.0,20.97,-0.165077,packaged foods,6.99,3,7,0,19
10843,0.92,3.0,44.97,-0.165077,meat,14.99,3,7,0,19


##Building the Model

In [293]:
#Encoding categorical features
categorical_cols = [cname for cname in merged_df.columns if merged_df[cname].dtype == "object"]
ordinal_encoder = OrdinalEncoder()
merged_df[categorical_cols] = ordinal_encoder.fit_transform(merged_df[categorical_cols])

# Split data into train and test data
X = merged_df.drop('estimated_stock_pct', axis=1)
y = merged_df['estimated_stock_pct']

In [294]:
#Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [295]:
X_train

,quantity,total,temperature,category,unit_price,month,day,day_of_week,hour
7258,0.0,0.0,-0.051368,6.0,16.99,3,5,5,16
5648,0.0,0.0,-0.729578,2.0,7.99,3,4,4,16
10241,0.0,0.0,-0.943848,7.0,5.49,3,7,0,15
6864,0.0,0.0,0.671020,14.0,5.49,3,5,5,13
9965,0.0,0.0,-0.105570,16.0,11.19,3,7,0,14
...,...,...,...,...,...,...,...,...,...
5734,0.0,0.0,-0.729578,14.0,5.49,3,4,4,16
5191,0.0,0.0,-1.150937,9.0,11.19,3,4,4,12
5390,0.0,0.0,-1.028733,12.0,20.49,3,4,4,14
860,0.0,0.0,0.337059,6.0,7.49,3,1,1,15


In [296]:
y_train

7258     0.125000
5648     0.393333
10241    0.570000
6864     0.570000
9965     0.665000
           ...   
5734     0.890000
5191     0.345000
5390     0.910000
860      0.890000
7270     0.205000
Name: estimated_stock_pct, Length: 8676, dtype: float64

##Model Building


In [297]:
#Linear Regression Model
regression=LinearRegression()
regression.fit(X_train,y_train)


LinearRegression()

In [298]:
#Prediction for the test data

reg_pred= regression.predict(X_test)

In [299]:
#Printing the coefficients and intercepts

print("Coefficients are", regression.coef_)
print("Intercept is", regression.intercept_)

Coefficients are [-9.19788542e-04  3.93926964e-04  2.52285254e-03  2.25308491e-05
 -1.25768511e-03  1.95156391e-18  2.33536429e-03 -2.31423115e-03
  7.89852223e-04]
Intercept is 0.5008675709786105


In [300]:
print("MAE is",mean_absolute_error(y_test,reg_pred))

MAE is 0.22200286945699718


In [301]:
#random Forest Model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

In [302]:
print("MAE is",mean_absolute_error(y_test,rf_pred))


MAE is 0.2347611873879412


In [303]:
DT_regressor = DecisionTreeRegressor()
DT_regressor.fit(X_train, y_train)
#Prediction for the test data
DT_pred = DT_regressor.predict(X_test)


In [304]:
print("MAE is",mean_absolute_error(y_test,DT_pred))

MAE is 0.30388704472106964


In [305]:
#XGBoost regressor Model
XGB = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
XGB.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_test, y_test)], verbose=False)

[08:02:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(learning_rate=0.05, n_estimators=1000, n_jobs=4)

In [306]:
#Prediction for the test data
XGB_pred = XGB.predict(X_test)

In [307]:
print("MAE is",mean_absolute_error(y_test,XGB_pred))

MAE is 0.22213642170624523
